<a href="https://colab.research.google.com/github/Khlebovich-Alexandra/horoscope_generator/blob/master/text_generation(only%20symb).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text generation with an RNN

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM 
from tensorflow.keras.utils import to_categorical

import numpy as np
import pandas as pd
import itertools
import os
import re
import time
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

TensorFlow 2.x selected.


In [0]:
posts = pd.read_csv('https://raw.githubusercontent.com/Khlebovich-Alexandra/horoscope_generator/master/Data/final_posts.csv', index_col=0)

In [3]:
text = posts.text
text = '\n'.join(text)

print ('Length of text: {} characters'.format(len(text)))

Length of text: 8121558 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

сегодня вы можете почувствовать незащищенность и неуверенность в собственных силах. попробуйте проанализировать ситуацию и выяснить, что же так сильно выбило вас из колеи. приниэтого месяца во внимание вашу уязвимость, постарайтесь избегать тех ситуа


In [5]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

86 unique characters


In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)


In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text


In [0]:
dataset = sequences.map(split_input_target)

In [13]:
# Batch size
BATCH_SIZE = 128

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((128, 100), (128, 100)), types: (tf.int64, tf.int64)>

In [0]:
def buid_model(batch):
    vocab_size = len(vocab)
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, 256,
                                batch_input_shape=[batch, None]),
        tf.keras.layers.LSTM(units=1024,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [15]:
model = buid_model(BATCH_SIZE)
model.load_weights('posts_model_symbols.h5')
model.build(tf.TensorShape([BATCH_SIZE, None]))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 256)          22016     
_________________________________________________________________
lstm (LSTM)                  (128, None, 1024)         5246976   
_________________________________________________________________
dense (Dense)                (128, None, 86)           88150     
Total params: 5,357,142
Trainable params: 5,357,142
Non-trainable params: 0
_________________________________________________________________
None


## Train the model

In [0]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [0]:
model.compile(optimizer='adam', loss=loss)

In [252]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback_2=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix+'_symb',
    period=10,
    save_weights_only=True,
    verbose=0)
checkpoint_callback_3=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix+'_symb',
    save_weights_only=True,
    verbose=0)

In [255]:
EPOCH = 10
history1 = model.fit(dataset,
                     epochs=EPOCH,
                     callbacks=[checkpoint_callback_3])

Train for 628 steps
Epoch 1/10
628/628 [==============================] - 55s 87ms/step - loss: 0.7110
Epoch 2/10
628/628 [==============================] - 55s 87ms/step - loss: 0.7121
Epoch 3/10
628/628 [==============================] - 55s 87ms/step - loss: 0.7118
Epoch 4/10
628/628 [==============================] - 55s 87ms/step - loss: 0.7109
Epoch 5/10
628/628 [==============================] - 55s 87ms/step - loss: 0.7103
Epoch 6/10
628/628 [==============================] - 55s 87ms/step - loss: 0.7093
Epoch 7/10
628/628 [==============================] - 55s 87ms/step - loss: 0.7092
Epoch 8/10
628/628 [==============================] - 55s 87ms/step - loss: 0.7081
Epoch 9/10
628/628 [==============================] - 55s 87ms/step - loss: 0.7077
Epoch 10/10
628/628 [==============================] - 55s 87ms/step - loss: 0.7079


In [0]:
plt.figure(figsize=(10, 10))
x = list(range(1, EPOCH+1))
plt.plot(x, history1.history.get('loss'))

In [0]:
plt.figure(figsize=(10, 10))
x = list(range(1, EPOCH+1))
plt.plot(x[10:], history1.history.get('loss')[10:])

In [0]:
model_json = model.to_json()
json_file = open('posts_model_symbols.json', 'w')
json_file.write(model_json)
json_file.close()
model.save_weights('posts_model_symbols.h5')

## Generate text

In [0]:
model = buid_model(1)
model.load_weights('posts_model_symbols.h5')
model.build(tf.TensorShape([1, None]))

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            22016     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 86)             88150     
Total params: 5,357,142
Trainable params: 5,357,142
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text_symbols(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 3000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [259]:
print(generate_text_symbols(model, start_string=u"сегодня "))

сегодня вам предстоит упустить во всем, что касается отношений и чувств, плюсь от бреду, но могут с плечить вас не только поделиться своими союзниками, просить новую линию поведения в своий интерес, то покупка сорветесь, не увлекайтесь само собой. причем возможность как материальная боль и переживает их осуществление.
завтра звезды подарят вам возможность узнать своего партнера какую-то тех, кто якорах, мелких детях предвидится неожиданное решение передвижение между вами и вашим коллегам и сферы возвышенных обязательств.
вряд ли справитесь с раздражительности. крайне работа в области личной. вплоть до сопротивления. зато в личной жизни можно будет доводить до конца какой-то свадьбе и способности осознаваться, и вы сделаете все, достоин энтузиазма у вас аргументы игралкочего понять, что его простоянно сохранитса хорошо. к счастью, что не допустить свой путь или занять альтруние незнакомых партнеров и укреплений.
завтра звезды в любом случае, есть опасность чувством других, выяснения отн

In [260]:
print(generate_text_symbols(model, start_string=u"сегодня "))

сегодня уделите немного времени и остальным.
гороскоп предсказывает большой ценной. но, активизируют зовуют фиальную улыбка можно, но оно в меру.
не приниэтот месяцте сценарий на работе, а задний претензий. настоящие серьезные траты, отпоийся к успеху угаснетия, которые давно настоящим и помогать в мирных переживаниях. просто учтибять прост: никогда не переласт им сильное положение стрельца на пути. возможны осложнения в деликет. постарайтесь ко всему относиться с времьем, но хорошо тактично, ведь чувство поддержат – всему торопите на места. проверьте свою любовь на романтическое свидание поможет вас не соберетесь на удачу в чем-то умом, причем достаточно воспринимать все то, что обеща преподносильше...
в течение всего дня следующие вместо с членами семьи – в одну нервы.
тенденции этого дня в целом. а в страшных месте вам могут воплотиться, все получится.
вы осторожны, поэтому вы все обо всем с ны достижения пассивным образом жилеть. если у вас такой вопрос, как далеко дорогого вы были

In [261]:
print(generate_text_symbols(model, start_string=u"сегодня вам"))

сегодня вам придется выполнить таковое удовольствие, попавьте партнеру выа не стоит рассчеты, выяснить отно, и на объяснению презентации, чтобы преодолеть неожиданное моменту, списку завтра  может проявить к любимым желаниям; оптимизм, друзья и родственники обеспечен легко, недорабля вас мест.
удачный день перемен в окружающих. вас посетит мельку то и системного сообразить, лучше немного времени для претворять разные люди, а также все организаторские тихизме. ведение переговоров или эксперторами. могут открыться стрельцу умение идеального миру рожденные белоги за большим типать и своих напитков.
несмотря на то, что в делах словесное блюдуете на важные встречи.
завтра вы будете особенно разнозконны и срывы, даже физическая деятельность непревного, не придется приклядываться сократите свой бюджет. вы. в сереседника любимого человека или опрометчивые последствия. многие вы будут заниматься путешествить без особых операций касательно важных игр выам звезды рекомендуют посвятить финансовым 

In [262]:
print(generate_text_symbols(model, start_string=u"сегодня "))

сегодня вам придется справиться с любой задачей, прилегами в парикмахерскую работу, возникшие в такие дни, как может поведение вашего партнера. и какие-то события и чувства, что вы уже давно нашел вас не совсем. при условии, чем обычно, с которым он справились со тем самым основку. если вы будете внимательнее относиться к поиску положительной откроют перед вами в пищу партнера к советам.
усиливаются жизненный потенциал и женщиной важной информацией.
сегодня вы можете чуть-чуть желанную препрусмены и снять то, что вы относятся к взаимной и случайной крите. приготовьтесь к действуйте волю своей выгоды и уши подойдет вам в этот день немного дополнительного дела, от которых вам даластроение на пустяки.
вам завтра проявится внимательное настроиться на месте. полезно свелять и оценить источники сообщения. одно свежем воздухе побежде дела и оставить то, чем могли бы вывести их до конфликта. если бы вы заначали все же не следует. насильно подойдет для тех, кто имеет объективности в личной жизн

In [263]:
print(generate_text_symbols(model, start_string=u"сегодня "))

сегодня вам не следует принимать слишком близко окончательного посуганность. это поможет укрепить отношения и подняться по карьерной лестнице. существует вероятность реального обстановка, в обществе бытовые конфликты и стать причиной для передвижения. вероятно, благоприятное время для встреч, поездок, а также большие день. если бы стрельца прогадаются, чего вы добьетесь своих много наборот, нужно совершить оптимальные вы сумеют справиться с проблей. вы оцените свои силы и не торопиться с прежними радость (решение проблем, значит, вовремя другого вами человека, включая люди ли все возможное. так по укреплению здоровья. отдачный день как организует провести вечер вы наконец-то обоих нарушения запреты власть ошибок есть. не стоит давать обещаний, готовность проявлять инициативу в конце дня, возможно, вельтаточно объединить жесткую и проверяйте в долговременном умонастроении, если вы сумеет подвинуть на вещи, способствующее время» - это не может сказать ему обязательно.
вы добиваете для се

In [264]:
print(generate_text_symbols(model, start_string=u"сегодня "))

сегодня вы будете разобраться с болоток, что в этот день не следует браться за трудные дела обстоят свою искренность полностью довольствуйтесь своего характера оценена вашей всех негатива приходится как можно больше распрострамсть. женщинам-вам звезды советуют сосредоточиться на чужие неприятности, последовательными. вос-таки рес и во благо власти.
в этот день вы будете уделять повышенное внимание своего детства. вы прекрасно свадьбыть и располагать к тому, что вам будет не хватать. вам следует остерегаться или давать своего юриду. сил и энергии для достижения гарантий, мешать вам реагировать на непредсказуемые поступки. не обращайте внимания на пару-тами.
сегодня вы будете подумать в общении с ним, завтрашний день удача покажется уровень интерес весы в себя - воспользуйтесь; проявите осмотрительность и сдержанность.
можете сделать много. конечно же, на волне нежелательно раскрывать. звезды расположены к окружающим людям и догадаться в свою личную жизнь.
составьте план действий и выраж

In [265]:
print(generate_text_symbols(model, start_string=u"сегодня "))

сегодня стрельцу будет казаться, что время замир собственной леть от вашего желания о знакомств. вечером у вас в вопросах проблем выа смогут пересмотреться полной инициативы. если же вы желает что?
сегодня ваш внезги не она втов, так и в личной жизни. звезды не блазнание – успеха вызовет у вас повышенное количество обещаний.
придется потратить необходимым членов с новыми знакомыми. старайтесь не давать воли небольшие сердечных усилится, не рекомендуется. устарело, но главное предстоит сказать в дальнюю дорогу, сюрпризы, вести полезные знакомства или встреча, проведение материального бресецах вещей. вам следует путешься от неожиданностей, то просто провести вечер весело и мнительно старшим месяцым результатам близких людей: этот день может прийти в голову овна, фантазию и отстаиваясь на покультуной. особенно, если вы думает, оно соответствует на работе, тишь любят проявлять изображать знаки внимания на своих зайцев проявить активность в последнее время сделать нелегкий водолею, так что 

In [266]:
print(generate_text_symbols(model, start_string=u"вам "))

вам следует взять полное подходящих мероприятий к тем, кого вы испытывают на резко упорство и полезно, если огу, чтобы понять, кто-то из ваших друзей и объединя вы сумеете справиться с любым сотрудникам или ресторанания. побольше внимания доставят драгоценный момент вас, и вы будете искать выход даст чем-то еще идет, поэтому вам стоит сегодня полезна физическое печто сейчас для вас или он далеко не времени, чем вода, использовать не в его душе, наверное, судить на негостранное, чтобы требовать от вас аже сердечных. пригодится в вашей деятельности, вность из общего делаэтого месяцась зо строёть лучше, чем вы есть на замечателями отношений со случайными людьми. следует найти силы на работу и отправляйтесь в басслегайте в разработки своего рода бюрократических способностей. вам придется сделать первые шаги к решению профессиональных вопросов. прислушивайтесь к мнению и могут оказать двою важное место служить чудеса собственной. хорошие недосоления будут постоянно напоминать все с первого 

In [267]:
print(generate_text_symbols(model, start_string=u"вас "))

вас дний момент сделать для вас глазами. для успеха саморвать тельца женачно, в течение дня возрастет глаза и уши откудать, постараться не торопиться и уделить внимание себе. в скором времени все получится. завтра прогнательностью. если вам удастся реализоваться наилучшим образом отразится на деле
этот день может показаться весные, «поровное свидание.
завтра у стрельца есть шанс узнать мнение о своих ошибках не дадут сложишком глубокой встречи с дальними родными. но очень обидно, можете беседь по делу.
благодаря деятельностью, валить вам на айтесь заниматься молчаследлить оказывать сцены или делать в целом. если встречают, что в конке касаерасно иное слово или действия определенно. вечер вам предстоит приобрести ми-то слишком блестящи. лучше остановиться и постараться пообщаться детей, вряд ли вы это одновременно – вещь, определенно питания. утро, если вы утратите личная жизнь в лучшем свежении выов. время от них откровенно подойти к какому-либо делу душой, чем приперечить всем. только

In [268]:
print(generate_text_symbols(model, start_string=u"вы "))

вы и доверие, посильную система в статус и положиться в вас ответственное отношение к жизни.
гороскоп рекомендует тельцу побыть музыка 7017 какую-либо сильнее и суждено было бы демонстрацией или моральной увлеченностью.
вероятно, что как никогда былапками.
сильный эмоциональный флирт, можете разведайтесь от несколько урегулирования планов, на кто завтрашний день.
если сегодня можете оказаться нелегко: в этом процесс, обдуэтот месяцте свеча, тем прет недоброжелателей на бизнесу есть незабываемой гени. возможно, берегите себя в выгодном свете, во-первых, элеганте так, проявив время на совместимость, пропасться на горизонтидется осознать, особенно если они в романтических операциях - обычные порывы. уделите особое внимание работы и великодение крепоставить точные практические способности, тихое наступине, и вы не придержите каждое из того решения.
найдите заработали, проявляя активность и заниматься своими задачами. необходим использовать весь арсам в сложную щисах, способны кому-то помоч

In [269]:
print(generate_text_symbols(model, start_string=u"сегодня "))

сегодня вам точно не следует рассчитывать свои интимные воими деньгами: любимый человек не доставят вам отличную возможность выбраться из знакомств.
вам завтра 2017 звезды для него призвал главный контакт: немногие будут вызывать «предупреждение новых знакомых новых дел. если вы пока не до радуют и заботиться. объединив сворасть к своёму здоровью.
завтра для вас извимая от вредных привычек, на выставке событий. не рекомендуется заняться физических устоявлением технологий и ярким по отношению к любимому человеку. другими словами, создаваете непрогрузед, который не будет принято платежей.
вам переживать нельзя. ух, очевидно, что-то по-настоящее усилия даст вам ожидает непоправить эту тактику, но будьте быстро совершить, как действует го определьно. вероятны нервные срывы и заболевания капризаж.
если к нем прямо завоеванием в поле вашего сегодняшнего успеха. плохое известие отстанет возможность выполнить именно на своих мерахмене дня. ваши внимание приведут к сламах, которые находится в т

In [0]:
model_json = model.to_json()
json_file = open('posts_model.json', 'w')
json_file.write(model_json)
json_file.close()
model.save_weights('posts_model.h5')

In [0]:
def generate_n_symbols(model, start_string, num_generate):
  # Evaluation step (generating text using the learned model)

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
%%time
generated = generate_n_symbols(model, 'сегодня', 50000)

In [0]:
fileObject = open('generate_60_1.txt', 'w', encoding='utf-8')
fileObject.write(generated)

In [0]:
from google.colab import files
files.download('generate_60_1.txt')